In [1]:
import os
import json
import pandas as pd
import seaborn as sns
from datetime import datetime
import numpy as np
import matplotlib.pylab as plt

In [2]:
### 1.1) SBA_Loan dataset cleaning and transformation for creating a model:

In [3]:
SBA_Loan = pd.read_csv('SBA_Loan_test.csv')

In [4]:
DF_SBA_Loan = SBA_Loan.copy()
DF_SBA_Loan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899164 entries, 0 to 899163
Data columns (total 28 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   LoanNr_ChkDgt      899164 non-null  int64  
 1   Name               899150 non-null  object 
 2   City               899134 non-null  object 
 3   State              899150 non-null  object 
 4   Zip                899164 non-null  int64  
 5   Bank               897605 non-null  object 
 6   BankState          897598 non-null  object 
 7   NAICS              899164 non-null  int64  
 8   ApprovalDate       899164 non-null  object 
 9   ApprovalFY         899164 non-null  int64  
 10  Term               899164 non-null  int64  
 11  NoEmp              899164 non-null  int64  
 12  NewExist           899028 non-null  float64
 13  CreateJob          899164 non-null  int64  
 14  RetainedJob        899164 non-null  int64  
 15  FranchiseCode      899164 non-null  int64  
 16  Ur

In [5]:
DF_SBA_Loan.isnull().sum()

LoanNr_ChkDgt             0
Name                     14
City                     30
State                    14
Zip                       0
Bank                   1559
BankState              1566
NAICS                     0
ApprovalDate              0
ApprovalFY                0
Term                      0
NoEmp                     0
NewExist                136
CreateJob                 0
RetainedJob               0
FranchiseCode             0
UrbanRural                0
RevLineCr              4528
LowDoc                 2582
ChgOffDate           736465
DisbursementDate       2368
DisbursementGross         0
BalanceGross              0
MIS_Status             1997
ChgOffPrinGr              0
GrAppv                    0
SBA_Appv                  0
Industry                  0
dtype: int64

In [6]:
# Filling MIS_Status NaN values
for x in DF_SBA_Loan[DF_SBA_Loan['MIS_Status'].isnull()]['ChgOffPrinGr'].index:
    if DF_SBA_Loan['ChgOffPrinGr'][x] == 0:
        DF_SBA_Loan.loc[x,'MIS_Status'] = "P I F"
    else:
        DF_SBA_Loan.loc[x,'MIS_Status'] = "CHGOFF"

In [7]:
DF_SBA_Loan.drop(DF_SBA_Loan[DF_SBA_Loan["NAICS"] == 0].index, inplace=True) # Dropping NAICS = 0 for now

In [8]:
#Keeping just the first 2 digits in the NAICS column of DF_SBA_Loan
NAICS_value = [str(number) for number in np.array(DF_SBA_Loan["NAICS"])]
NAICS_first_two = [s[:2] for s in NAICS_value]
DF_SBA_Loan["NAICS"] = [int(x) for x in NAICS_first_two]

In [9]:
DF_SBA_Loan = DF_SBA_Loan[['State', 'NAICS', 'ApprovalFY', 'NewExist', 'MIS_Status', 'SBA_Appv', 'GrAppv']] #'UrbanRural', 'LowDoc' left out
DF_SBA_Loan = DF_SBA_Loan.reset_index(drop=True)
DF_SBA_Loan.drop(DF_SBA_Loan[DF_SBA_Loan['NewExist'] == 0].index, inplace=True)

In [10]:
DF_SBA_Loan = DF_SBA_Loan.dropna()

In [11]:
DF_SBA_Loan.head()

,State,NAICS,ApprovalFY,NewExist,MIS_Status,SBA_Appv,GrAppv
0,IN,45,1997,2.0,P I F,48000.0,60000.0
1,IN,72,1997,2.0,P I F,32000.0,40000.0
2,IN,62,1997,1.0,P I F,215250.0,287000.0
3,CT,33,1997,1.0,P I F,387750.0,517000.0
4,FL,81,1997,2.0,P I F,36000.0,45000.0


In [12]:
# DF_SBA_Loan['UrbanRural'].value_counts()
# DF_SBA_Loan[DF_SBA_Loan['UrbanRural'] == 0]
# sns.countplot(x='NAICS', data=DF_SBA_Loan, hue='NewExist')
# DF_SBA_Loan['MIS_Status'].value_counts()

In [13]:
### 1.2) NAICS_BDS dataset cleaning and transformation for creating a new features for our model:

In [14]:
NAICS_BDS = pd.read_csv('NAICS_BDS_timeseries_data.csv', low_memory=False)
NAICS_BDS = NAICS_BDS.drop([0])
NAICS_BDS.head()

,GEO_ID,NAME,NAICS,NAICS_LABEL,YEAR,FIRM,ESTAB,EMP,DENOM,ESTABS_ENTRY,...,JOB_DESTRUCTION_DEATHS,JOB_DESTRUCTION_CONTINUERS,JOB_DESTRUCTION_RATE_DEATHS,JOB_DESTRUCTION_RATE,NET_JOB_CREATION,NET_JOB_CREATION_RATE,REALLOCATION_RATE,FIRMDEATH_FIRMS,FIRMDEATH_ESTABS,FIRMDEATH_EMP
1,0100000US,United States,0,Total for all sectors,1978,3557994,4310626,69410001,66810001,654226,...,4048046,5420961,6.059,14.173,5196537,7.778,28.346,327278,332504,1824251
2,0100000US,United States,0,Total for all sectors,1979,3692077,4472108,73848234,71727111,630253,...,3760616,6043218,5.243,13.668,4248257,5.923,27.336,328024,332633,1785478
3,0100000US,United States,0,Total for all sectors,1980,3739809,4533251,74109267,74018218,592484,...,4246135,7797198,5.737,16.271,188937,0.255,32.542,368578,373754,1941422
4,0100000US,United States,0,Total for all sectors,1981,3770852,4615479,75728652,74924061,606853,...,3478843,7782399,4.643,15.03,1609104,2.148,30.06,377291,383748,2005707
5,0100000US,United States,0,Total for all sectors,1982,3720273,4598769,74922226,75267274,572030,...,4055239,8683755,5.388,16.925,-678350,-0.901,32.048,412488,418501,2211944


In [15]:
DF_NAICS = NAICS_BDS.copy()
DF_NAICS.drop(DF_NAICS[(DF_NAICS.NAICS == '0')].index, inplace=True) #Dropping NAICS = 0 as it is the total for all the categories year wise

In [16]:
# Transforming NAICS range column to individual columns:
DF_NAICS.drop(DF_NAICS[(DF_NAICS.NAICS == '31-33') | (DF_NAICS.NAICS == '44-45') | (DF_NAICS.NAICS == '48-49')].index, inplace=True)
DF_NAICS = DF_NAICS.append([NAICS_BDS[NAICS_BDS["NAICS"] == '31-33']]*3, ignore_index=True)
DF_NAICS = DF_NAICS.append([NAICS_BDS[NAICS_BDS["NAICS"] == '44-45']]*2, ignore_index=True)
DF_NAICS = DF_NAICS.append([NAICS_BDS[NAICS_BDS["NAICS"] == '48-49']]*2, ignore_index=True)
# Renaming the NAICS entries of range-columns
temp_array = [31]*42 + [32]*42 + [33]*42
DF_NAICS = DF_NAICS.replace({'NAICS' : { '31-33' : temp_array}})
temp_array = [44]*42 + [45]*42
DF_NAICS = DF_NAICS.replace({'NAICS' : { '44-45' : temp_array}})
temp_array = [48]*42 + [49]*42
DF_NAICS = DF_NAICS.replace({'NAICS' : { '48-49' : temp_array}})

In [17]:
DF_NAICS.head()

,GEO_ID,NAME,NAICS,NAICS_LABEL,YEAR,FIRM,ESTAB,EMP,DENOM,ESTABS_ENTRY,...,JOB_DESTRUCTION_DEATHS,JOB_DESTRUCTION_CONTINUERS,JOB_DESTRUCTION_RATE_DEATHS,JOB_DESTRUCTION_RATE,NET_JOB_CREATION,NET_JOB_CREATION_RATE,REALLOCATION_RATE,FIRMDEATH_FIRMS,FIRMDEATH_ESTABS,FIRMDEATH_EMP
0,0100000US,United States,11,"Agriculture, forestry, fishing and hunting",1978,23763,24429,200156,192326,4717,...,18829,27500,9.79,24.089,15439,8.028,48.178,2876,2905,12497
1,0100000US,United States,11,"Agriculture, forestry, fishing and hunting",1979,21765,22373,192288,191325,4178,...,22142,30446,11.573,27.486,1716,0.897,54.972,3022,3068,14659
2,0100000US,United States,11,"Agriculture, forestry, fishing and hunting",1980,21882,22500,193997,193979,3827,...,16453,31944,8.482,24.95,-108,-0.056,49.788,2780,2780,11264
3,0100000US,United States,11,"Agriculture, forestry, fishing and hunting",1981,21862,22468,189977,193065,3514,...,20576,31625,10.658,27.038,-6316,-3.271,47.533,2696,2699,11719
4,0100000US,United States,11,"Agriculture, forestry, fishing and hunting",1982,19040,19695,173515,178279,2775,...,20541,31150,11.522,28.995,-9420,-5.284,47.421,3176,3180,14178


In [18]:
Keep_Columns_DF_NAICS = ['NAICS', 'YEAR', 'FIRM', 'DENOM', 'ESTABS_ENTRY', 'ESTABS_EXIT', 'JOB_CREATION', 'JOB_DESTRUCTION', 'REALLOCATION_RATE', 'FIRMDEATH_ESTABS', 'FIRMDEATH_EMP']
DF_NAICS = DF_NAICS[Keep_Columns_DF_NAICS]

In [19]:
To_int_Columns_DF_NAICS = ['NAICS', 'YEAR', 'FIRM', 'DENOM', 'ESTABS_ENTRY', 'ESTABS_EXIT', 'JOB_CREATION', 'JOB_DESTRUCTION', 'FIRMDEATH_ESTABS', 'FIRMDEATH_EMP']
for column_name in To_int_Columns_DF_NAICS:
    DF_NAICS[column_name] = [int(x) for x in DF_NAICS[column_name]]

DF_NAICS['REALLOCATION_RATE'] = [float(x) for x in DF_NAICS['REALLOCATION_RATE']]

In [20]:
### 1.3) Merging Dataset based on NAICS category and dates: Year from DF_NAICS and ApprovalFY from DF_SBA_Loan
DF_SBA_Loan = DF_SBA_Loan.rename(columns={"ApprovalFY": "YEAR"})
print(DF_SBA_Loan.shape)
print(DF_NAICS.shape)

(696342, 7)
(966, 11)


In [21]:
df_merged_NAICS_SBALoan = pd.merge(DF_SBA_Loan, DF_NAICS,  how='left', left_on=['NAICS','YEAR'], right_on = ['NAICS','YEAR'])
df_merged_NAICS_SBALoan = df_merged_NAICS_SBALoan.dropna()
df_merged_NAICS_SBALoan

,State,NAICS,YEAR,NewExist,MIS_Status,SBA_Appv,GrAppv,FIRM,DENOM,ESTABS_ENTRY,ESTABS_EXIT,JOB_CREATION,JOB_DESTRUCTION,REALLOCATION_RATE,FIRMDEATH_ESTABS,FIRMDEATH_EMP
0,IN,45,1997,2.0,P I F,48000.0,60000.0,670945.0,13763872.0,110091.0,115364.0,2361731.0,2038489.0,29.621,73384.0,349798.0
1,IN,72,1997,2.0,P I F,32000.0,40000.0,357519.0,9152783.0,69257.0,63282.0,1894562.0,1728105.0,37.761,45776.0,441164.0
2,IN,62,1997,1.0,P I F,215250.0,287000.0,481657.0,12684590.0,60737.0,50485.0,1943403.0,1632957.0,25.747,35763.0,328510.0
3,CT,33,1997,1.0,P I F,387750.0,517000.0,292577.0,16284152.0,28296.0,29516.0,1835979.0,1672272.0,20.539,22142.0,255029.0
4,FL,81,1997,2.0,P I F,36000.0,45000.0,599136.0,4864066.0,62099.0,58638.0,800963.0,694752.0,28.567,42290.0,156087.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696337,MD,33,1997,1.0,P I F,25000.0,50000.0,292577.0,16284152.0,28296.0,29516.0,1835979.0,1672272.0,20.539,22142.0,255029.0
696338,CA,31,1997,1.0,P I F,150000.0,200000.0,292577.0,16284152.0,28296.0,29516.0,1835979.0,1672272.0,20.539,22142.0,255029.0
696339,OH,45,1997,1.0,P I F,56000.0,70000.0,670945.0,13763872.0,110091.0,115364.0,2361731.0,2038489.0,29.621,73384.0,349798.0
696340,OH,45,1997,1.0,P I F,42500.0,85000.0,670945.0,13763872.0,110091.0,115364.0,2361731.0,2038489.0,29.621,73384.0,349798.0


In [22]:
# Plotting correlation matrix
correlation = df_merged_NAICS_SBALoan.corr()
# plt.figure(figsize=(10,10))
# plt.title('Correlation Heatmap of NAICS_SBA_Dataset')
# ax = sns.heatmap(correlation, square=True, annot=True, fmt='.2f', linecolor='white')
# ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
# ax.set_yticklabels(ax.get_yticklabels(), rotation=30)
# plt.show()

In [23]:
### 1.4) Splitting the data into Train, Val and Test as per time-series:
df_merged_NAICS_SBALoan_train = df_merged_NAICS_SBALoan[df_merged_NAICS_SBALoan['YEAR']<=2007]
df_merged_NAICS_SBALoan_val = df_merged_NAICS_SBALoan[(df_merged_NAICS_SBALoan['YEAR'] == 2008) | (df_merged_NAICS_SBALoan['YEAR'] == 2009)]
df_merged_NAICS_SBALoan_test = df_merged_NAICS_SBALoan[df_merged_NAICS_SBALoan['YEAR'] >= 2010]

In [24]:
y_train = df_merged_NAICS_SBALoan_train['SBA_Appv'] / df_merged_NAICS_SBALoan_train['GrAppv']
X_train = df_merged_NAICS_SBALoan_train.drop(['SBA_Appv'], axis=1)
y_val = df_merged_NAICS_SBALoan_val['SBA_Appv'] / df_merged_NAICS_SBALoan_val['GrAppv']
X_val = df_merged_NAICS_SBALoan_val.drop(['SBA_Appv'], axis=1)
y_test = df_merged_NAICS_SBALoan_test['SBA_Appv'] / df_merged_NAICS_SBALoan_test['GrAppv']
X_test = df_merged_NAICS_SBALoan_test.drop(['SBA_Appv'], axis=1)

In [25]:
from sklearn.preprocessing import StandardScaler

# Define numerical features to use for modelling
num_features = ['GrAppv', 'FIRM', 'DENOM', 'ESTABS_ENTRY', 'ESTABS_EXIT', 'JOB_CREATION', 'JOB_DESTRUCTION', 'REALLOCATION_RATE', 'FIRMDEATH_ESTABS', 'FIRMDEATH_EMP']

# Define categorical features to use for modelling
cat_features = ['State', 'NAICS', 'MIS_Status']
misc_features = ['YEAR', 'NewExist']

# Fitting scaler for the training data
scaler = StandardScaler()
scaler.fit(X_train[num_features])

def features_scale_and_dummies(df):
    scaled = df[num_features].copy()
    # Convert numeric features to standard units
    scaled.iloc[:, :] = scaler.transform(scaled)
    # Convert categorical features using dummy encoding
    categoricals = [pd.get_dummies(df[s], prefix=s, drop_first=True) for s in cat_features]
    misc = df[misc_features]
    return pd.concat([scaled] + categoricals + [misc], axis=1)

In [26]:
X_train = features_scale_and_dummies(X_train)
X_val = features_scale_and_dummies(X_val)
X_test = features_scale_and_dummies(X_test)

In [27]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(599434, 85)
(58648, 85)
(38031, 85)


In [28]:
#Defining RMSE function
from sklearn import metrics
def rmse(y_pred, y):
    return np.sqrt(metrics.mean_squared_error(y_pred, y))
from sklearn.metrics import r2_score

In [29]:
### 1.4.1) Considering only Entering Entities features to train our model:
# Columns_to_keep = ['State', 'NAICS', 'YEAR', 'NewExist', 'MIS_Status', 'GrAppv', 'FIRM', 'DENOM', 'ESTABS_ENTRY', 'JOB_CREATION']
X_train_1 = X_train.drop(['ESTABS_EXIT', 'JOB_DESTRUCTION', 'REALLOCATION_RATE', 'FIRMDEATH_ESTABS', 'FIRMDEATH_EMP'], axis=1)
X_val_1 = X_val.drop(['ESTABS_EXIT', 'JOB_DESTRUCTION', 'REALLOCATION_RATE', 'FIRMDEATH_ESTABS', 'FIRMDEATH_EMP'], axis=1)

In [30]:
# Linear Regression - Performing Linear Regression
from sklearn.linear_model import LinearRegression
model_1 = LinearRegression(fit_intercept=True).fit(X_train_1, y_train)
y_train_predicted_1 = model_1.predict(X_train_1)
y_val_predicted_1 = model_1.predict(X_val_1)
# Linear Regression - RMSE for training and validation data
rmse_train_1 = rmse(y_train, y_train_predicted_1)
rmse_val_1 = rmse(y_val, y_val_predicted_1)
r2_train_1 = r2_score(y_train, y_train_predicted_1)
r2_val_1 = r2_score(y_val, y_val_predicted_1)
print("For the linear regression model in part 1:\n", "RMSE for training data is: ",rmse_train_1,"\n","RMSE for validation data is: ",rmse_val_1,"\n","R2 Score for training data is: ",r2_train_1,"\n","R2 Score for validation data is: ",r2_val_1)

For the linear regression model in part 1:
 RMSE for training data is:  0.13823452738522352 
 RMSE for validation data is:  0.181954968933335 
 R2 Score for training data is:  0.3644253807312253 
 R2 Score for validation data is:  0.014596587208754563


In [31]:
# # Plotting residual Curves for the Regression model
# residuals_1 = y_val - y_val_predicted_1
# ax = sns.regplot(y_val, residuals_1)
# ax.set_xlabel('SBA Approved / Gross Approved (Validation Data)')
# ax.set_ylabel('Residuals (Actual Price - Predicted Price)')
# ax.set_title("Residuals vs. SBA Approved / Gross Approved on Validation Data");

In [32]:
### 1.4.2) Considering only Exiting Entities features to train our model:
# Columns_to_keep = ['State', 'NAICS', 'YEAR', 'NewExist', 'MIS_Status', 'GrAppv', 'FIRM', 'DENOM', 'ESTABS_EXIT', 'JOB_DESTRUCTION', 'REALLOCATION_RATE', 'FIRMDEATH_ESTABS', 'FIRMDEATH_EMP']
X_train_2 = X_train.drop(['ESTABS_ENTRY', 'JOB_CREATION'], axis=1)
X_val_2 = X_val.drop(['ESTABS_ENTRY', 'JOB_CREATION'], axis=1)

In [33]:
# Linear Regression - Performing Linear Regression
model_2 = LinearRegression(fit_intercept=True).fit(X_train_2, y_train)
y_train_predicted_2 = model_2.predict(X_train_2)
y_val_predicted_2 = model_2.predict(X_val_2)
# Linear Regression - RMSE for training and validation data
rmse_train_2 = rmse(y_train, y_train_predicted_2)
rmse_val_2 = rmse(y_val, y_val_predicted_2)
r2_train_2 = r2_score(y_train, y_train_predicted_2)
r2_val_2 = r2_score(y_val, y_val_predicted_2)
print("For the linear regression model in part 2:\n", "RMSE for training data is: ",rmse_train_2,"\n","RMSE for validation data is: ",rmse_val_2,"\n","R2 Score for training data is: ",r2_train_2,"\n","R2 Score for validation data is: ",r2_val_2)

For the linear regression model in part 2:
 RMSE for training data is:  0.13815035384295604 
 RMSE for validation data is:  0.18300784141692794 
 R2 Score for training data is:  0.36519917118663214 
 R2 Score for validation data is:  0.003159627007398691


In [34]:
# # Plotting residual Curves for the Regression model
# residuals_2 = y_val - y_val_predicted_2
# ax = sns.regplot(y_val, residuals_2)
# ax.set_xlabel('SBA Approved / Gross Approved (Validation Data)')
# ax.set_ylabel('Residuals (Actual Price - Predicted Price)')
# ax.set_title("Residuals vs. SBA Approved / Gross Approved on Validation Data");

In [35]:
### 1.4.3) Considering all the features to train our model:
# Linear Regression - Performing Linear Regression
model_3 = LinearRegression(fit_intercept=True).fit(X_train, y_train)
y_train_predicted_3 = model_3.predict(X_train)
y_val_predicted_3 = model_3.predict(X_val)
# Linear Regression - RMSE for training and validation data
rmse_train_3 = rmse(y_train, y_train_predicted_3)
rmse_val_3 = rmse(y_val, y_val_predicted_3)
r2_train_3 = r2_score(y_train, y_train_predicted_3)
r2_val_3 = r2_score(y_val, y_val_predicted_3)
print("For the linear regression model in part 3:\n", "RMSE for training data is: ",rmse_train_3,"\n","RMSE for validation data is: ",rmse_val_3,"\n","R2 Score for training data is: ",r2_train_3,"\n","R2 Score for validation data is: ",r2_val_3)

For the linear regression model in part 3:
 RMSE for training data is:  0.138148359102862 
 RMSE for validation data is:  0.18247831297607567 
 R2 Score for training data is:  0.3652175027143919 
 R2 Score for validation data is:  0.008919944779136935


In [36]:
# # Plotting residual Curves for the Regression model
# residuals_3 = y_val - y_val_predicted_3
# ax = sns.regplot(y_val, residuals_3)
# ax.set_xlabel('SBA Approved / Gross Approved (Validation Data)')
# ax.set_ylabel('Residuals (Actual Price - Predicted Price)')
# ax.set_title("Residuals vs. SBA Approved / Gross Approved on Validation Data");

In [37]:
### 1.5) Splitting the data into Train, Val and Test as per time-series:
df_merged_NAICS_SBALoan_train_val = df_merged_NAICS_SBALoan[df_merged_NAICS_SBALoan['YEAR']<=2009]
df_merged_NAICS_SBALoan_test = df_merged_NAICS_SBALoan[df_merged_NAICS_SBALoan['YEAR'] >= 2010]
y_train_val = df_merged_NAICS_SBALoan_train_val['SBA_Appv'] / df_merged_NAICS_SBALoan_train_val['GrAppv']
X_train_val = df_merged_NAICS_SBALoan_train_val.drop(['SBA_Appv'], axis=1)
y_test = df_merged_NAICS_SBALoan_test['SBA_Appv'] / df_merged_NAICS_SBALoan_test['GrAppv']
X_test = df_merged_NAICS_SBALoan_test.drop(['SBA_Appv'], axis=1)
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.15, random_state= 0)

In [38]:
# Fitting scaler for the new training data
scaler = StandardScaler()
scaler.fit(X_train[num_features])
# Redefining the function with new Scaler
def features_scale_and_dummies(df):
    scaled = df[num_features].copy()
    # Convert numeric features to standard units
    scaled.iloc[:, :] = scaler.transform(scaled)
    # Convert categorical features using dummy encoding
    categoricals = [pd.get_dummies(df[s], prefix=s, drop_first=True) for s in cat_features]
    misc = df[misc_features]
    return pd.concat([scaled] + categoricals + [misc], axis=1)

In [39]:
X_train = features_scale_and_dummies(X_train)
X_val = features_scale_and_dummies(X_val)
X_test = features_scale_and_dummies(X_test)

In [40]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(559369, 85)
(98713, 85)
(38031, 85)


In [41]:
# Linear Regression - Performing Linear Regression
model_3 = LinearRegression(fit_intercept=True).fit(X_train, y_train)
y_train_predicted_3 = model_3.predict(X_train)
y_val_predicted_3 = model_3.predict(X_val)
# Linear Regression - RMSE for training and validation data
rmse_train_3 = rmse(y_train, y_train_predicted_3)
rmse_val_3 = rmse(y_val, y_val_predicted_3)
r2_train_3 = r2_score(y_train, y_train_predicted_3)
r2_val_3 = r2_score(y_val, y_val_predicted_3)
print("For the linear regression model in part 3:\n", "RMSE for training data is: ",rmse_train_3,"\n","RMSE for validation data is: ",rmse_val_3,"\n","R2 Score for training data is: ",r2_train_3,"\n","R2 Score for validation data is: ",r2_val_3)

For the linear regression model in part 3:
 RMSE for training data is:  0.14164927178163114 
 RMSE for validation data is:  0.14192099845032854 
 R2 Score for training data is:  0.34176446266767035 
 R2 Score for validation data is:  0.3392913497046307


In [42]:
# # Plotting residual Curves for the Regression model
# residuals_3 = y_val - y_val_predicted_3
# ax = sns.regplot(y_val, residuals_3)
# ax.set_xlabel('SBA Approved / Gross Approved (Validation Data)')
# ax.set_ylabel('Residuals (Actual Price - Predicted Price)')
# ax.set_title("Residuals vs. SBA Approved / Gross Approved on Validation Data");

In [43]:
####TRASH#####

In [44]:
# ###Polynomial Regression
# from sklearn.preprocessing import PolynomialFeatures
# poly = PolynomialFeatures()
# poly.fit(X_train_2)
# X_train_2_poly = poly.transform(X_train_2)
# X_val_2_poly = poly.transform(X_val_2)

In [45]:
# Linear Regression - Performing Linear Regression
# poly_model_2 = LinearRegression(fit_intercept=True).fit(X_val_2_poly, y_train)
# y_train_predicted_poly_2 = poly_model_2.predict(X_train_2_poly)
# y_val_predicted_poly_2 = poly_model_2.predict(X_val_2_poly)
# # Linear Regression - RMSE for training and validation data
# rmse_train_poly_2 = rmse(y_train, y_train_predicted_poly_2)
# rmse_val_poly_2 = rmse(y_val, y_val_predicted_poly_2)
# print("For the linear regression model in part 2:\n", "RMSE for training data is: ",rmse_train_poly_2,"\n","RMSE for validation data is: ",rmse_val_poly_2)

In [46]:
# ###PCA
# from sklearn.decomposition import PCA
# pca = PCA(n_components=85, random_state = 0)
# pca.fit(X_train)

In [47]:
# from sklearn import preprocessing
# X_train_pca = pca.transform(preprocessing.scale(X_train))
# X_train_pca = pd.DataFrame(X_train_pca)
# X_val_pca = pca.transform(preprocessing.scale(X_val))
# X_val_pca = pd.DataFrame(X_val_pca)

In [48]:
# variance = pca.explained_variance_
# prop_variance = pca.explained_variance_ratio_
# cum_prop_variance = np.cumsum(pca.explained_variance_ratio_)
# pca_summary = pd.DataFrame({'Variance':variance, 'Proportion of Variance':prop_variance, 'Cumulative Proportion':cum_prop_variance}).T
# pca_column_name_array = []
# for num in range(1,86):
#     name='PC'+str(num)
#     pca_column_name_array.append(name)
# pca_summary.columns = pca_column_name_array
# pca_summary

In [49]:
# plt.figure(figsize=(7,4))
# plt.plot(pca_summary.iloc[1:2].T.loc["PC1":"PC10"]*100, marker = 'o', linestyle= '-')
# plt.plot(pca_summary.iloc[2:3].T.loc["PC1":"PC10"]*100, marker = 'o', linestyle= '-')
# plt.legend(['Variance', 'Cummulative Variance'])
# plt.xlabel('Principal Component', fontsize = 14)
# plt.ylabel('Proportion of Explained Variance, %', fontsize = 14);

In [50]:
# Linear Regression - Performing Linear Regression
# model_4 = LinearRegression(fit_intercept=True).fit(X_train_pca.iloc[: , :10], y_train)
# # y_train_predicted_4 = model_4.predict(X_train_pca.iloc[: , :10])
# y_val_predicted_4 = model_4.predict(X_val_pca.iloc[: , :10])
# # Linear Regression - RMSE for training and validation data
# rmse_train_4 = rmse(y_train, y_train_predicted_4)
# rmse_val_4 = rmse(y_val, y_val_predicted_4)
# print("For the linear regression model in part 4:\n", "RMSE for training data is: ", rmse_train_4, "\n","RMSE for validation data is: ", rmse_val_4)